In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from dataset.sr_dataset import DatasetSR
from PIL import Image as im

from models.EDSR import EDSR
import torch
import torch.nn as nn
from utils.utils import calculate_psnr,calculate_ssim
from tqdm import tqdm
from dataset.sr_dataset import DatasetSR
from einops import rearrange
from torch.utils.data import DataLoader
import cv2
import glob
import os

In [5]:
weights=glob.glob('experiment/EDSR/*.pt')

In [21]:
GT_img_path=glob.glob('/home/joowan/Desktop/dataset/Set5/Set5/GTmod12/*.png')
LR_img_path=glob.glob('/home/joowan/Desktop/dataset/Set5/Set5/LRbicx2/*.png')
print(GT_img_path)
print(LR_img_path)

['/home/joowan/Desktop/dataset/Set5/Set5/GTmod12/butterfly.png', '/home/joowan/Desktop/dataset/Set5/Set5/GTmod12/woman.png', '/home/joowan/Desktop/dataset/Set5/Set5/GTmod12/head.png', '/home/joowan/Desktop/dataset/Set5/Set5/GTmod12/bird.png', '/home/joowan/Desktop/dataset/Set5/Set5/GTmod12/baby.png']
['/home/joowan/Desktop/dataset/Set5/Set5/LRbicx2/butterfly.png', '/home/joowan/Desktop/dataset/Set5/Set5/LRbicx2/woman.png', '/home/joowan/Desktop/dataset/Set5/Set5/LRbicx2/head.png', '/home/joowan/Desktop/dataset/Set5/Set5/LRbicx2/bird.png', '/home/joowan/Desktop/dataset/Set5/Set5/LRbicx2/baby.png']


In [34]:
device='cuda:0' if torch.cuda.is_available() else 'cpu'
model=EDSR(scale=2)
model.to(device)

for weight_path in weights:

    weight=torch.load(weight_path)
    model.load_state_dict(weight)

    current_loss=0
    psnr=0
    ssim=0
    criterion=nn.L1Loss()
    with torch.no_grad():
        for i in range(len(GT_img_path)):
            LR_img=torch.from_numpy(cv2.cvtColor(cv2.imread(LR_img_path[i]),cv2.COLOR_BGR2RGB)).permute(2,0,1).float()
            HR_img=torch.from_numpy(cv2.cvtColor(cv2.imread(GT_img_path[i]),cv2.COLOR_BGR2RGB)).permute(2,0,1).float()
            
            LR_img=LR_img.unsqueeze(0).to(device)
            HR_img=HR_img.unsqueeze(0).to(device)
            
            output_img=model(LR_img)
            current_loss+=criterion(output_img,HR_img)
            
            output_img=output_img[0].cpu().numpy()
            HR_img=HR_img[0].cpu().numpy()
            output_img=(rearrange(output_img,'c h w -> h w c'))
            HR_img=(rearrange(HR_img,'c h w -> h w c'))

            psnr+=calculate_psnr(output_img,HR_img,crop_border=8)
            ssim+=calculate_ssim(output_img,HR_img,crop_border=8)
            
            
            #####################
            # save result image #
            #####################
            output_img=cv2.cvtColor(output_img,cv2.COLOR_BGR2RGB)
            HR_img=cv2.cvtColor(HR_img,cv2.COLOR_BGR2RGB)
            LR_img=LR_img[0].cpu().numpy()
            LR_img=(rearrange(LR_img,'c h w -> h w c'))
            LR_img=cv2.cvtColor(LR_img,cv2.COLOR_BGR2RGB)
            LR_img=cv2.resize(LR_img,(output_img.shape[1],output_img.shape[0]),)
            
            # cv2.imwrite(f'SR_{idx}.png',output_img)
            # cv2.imwrite(f'HR_{idx}.png',HR_img)
            # cv2.imwrite(f'LR_{idx}.png',LR_img)
            
            
        epoch_loss=current_loss / len(GT_img_path)
        avg_psnr=psnr / len(GT_img_path)
        avg_ssim=ssim / len(GT_img_path)
        
        print(weight_path) 
        print(avg_psnr)
        print(avg_ssim)
    
    

experiment/EDSR/psnr_best.pt
32.95438004384114
0.9250629109920314
experiment/EDSR/1_best.pt
32.96284397009521
0.9251011496345022
experiment/EDSR/2_best.pt
32.95293615387901
0.9252083236635981
experiment/EDSR/loss_best.pt
32.95438004384114
0.9250629109920314
